In [1]:
import pandas as pd
price_df = pd.read_csv('ETL_Final.csv')
price_df.shape

(6860645, 32)

In [2]:
price_df['Postcode'] = price_df['Postcode'].str.replace(' ', '').str.upper()
price_df.head()

,Transfer Date,Postcode,primary_addressable_object_name,secondary_addressable_object_name,street,Town/City,District,County,Price (Thousands),Freehold Tenure,...,Postcode Area,Postcode District,Area Code Average Price (Thousands),Town/City Average Price (Thousands),District Average Price (Thousands),County Average Price (Thousands),LAT,LONG,OSEAST1M,OSNRTH1M
0,2018-09-24,M68GQ,1,NaN,RIVINGTON ROAD,SALFORD,SALFORD,GREATER MANCHESTER,253.500,1,...,M,M6,286.373466,274.417259,259.866315,264.161265,53.49295,-2.311376,379439.0,399626.0
1,2018-09-28,WA32UE,35,NaN,STONEACRE CLOSE,WARRINGTON,WIGAN,GREATER MANCHESTER,231.950,1,...,WA,WA3,301.647233,295.472854,194.319863,264.161265,53.46955,-2.583977,361332.0,397136.0
2,2018-08-29,OL66RJ,102,NaN,THORNFIELD GROVE,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,112.500,1,...,OL,OL6,197.639388,184.326053,196.955406,264.161265,53.48984,-2.074338,395165.0,399237.0
3,2018-06-15,M460TW,37,NaN,THREADNEEDLE PLACE,MANCHESTER,WIGAN,GREATER MANCHESTER,184.995,1,...,M,M46,286.373466,288.415706,194.319863,264.161265,53.52589,-2.495181,367270.0,403360.0
4,2018-09-28,M283XS,9,NaN,MARPLE GARDENS,MANCHESTER,SALFORD,GREATER MANCHESTER,214.995,0,...,M,M28,286.373466,288.415706,259.866315,264.161265,53.51677,-2.411523,372810.0,402310.0


In [13]:
file_path = "matched_epc_rows/domestic-E06000001-Hartlepool.parquet"
df = pd.read_parquet(file_path)

# Print the first 5 rows
print(df.head())

                              LMK_KEY               ADDRESS1 ADDRESS2  \
0  1451865729102016060917325243560118       54, Chatham Road     None   
1   978835396212019052007505697910817    19, Wainwright Walk     None   
2  1341712688132019081910310554978408       116, Milton Road     None   
3  1186183659802014081111373821640768  18, Brafferton Street     None   
4  1566486539802017081008512757330418     132, Kesteven Road     None   

  ADDRESS3  POSTCODE  BUILDING_REFERENCE_NUMBER CURRENT_ENERGY_RATING  \
0     None  TS24 8QQ                  134825478                     D   
1     None  TS25 1XA                 8777671178                     D   
2     None  TS26 8DX                 8864447378                     D   
3     None  TS26 8LH                 4895056278                     D   
4     None  TS25 2NJ                  862833578                     C   

  POTENTIAL_ENERGY_RATING  CURRENT_ENERGY_EFFICIENCY  \
0                       B                         61   
1         

In [109]:
import os
import pandas as pd
import re
from tqdm import tqdm


# --- Normalization Function ---
def normalize_address_components(*components):
    combined = ' '.join([str(c) for c in components if pd.notna(c)])
    normalized = (
        combined.upper()
        .replace(',', '')
        .replace('.', '')
        .replace('-', ' ')
    )
    normalized = re.sub(r'\s+', ' ', normalized)
    return normalized.strip()

# --- Normalize price_df ---
price_df['postcode_clean'] = price_df['Postcode'].astype(str).str.replace(' ', '').str.upper()
price_df['match_key'] = price_df.apply(
    lambda row: normalize_address_components(
        row.get('primary_addressable_object_name', ''),
        row.get('street', ''),
        row['postcode_clean']
    ),
    axis=1
)

# --- EPC Folder Loop with Progress ---
folder = 'epc_certificates_only'
subfolders = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]

output_dir = "matched_epc_rows"
os.makedirs(output_dir, exist_ok=True)

for subfolder in tqdm(subfolders, desc="Processing EPC folders", unit="file"):
    cert_path = os.path.join(folder, subfolder, 'certificates.csv')

    if not os.path.exists(cert_path):
        continue

    try:
        epc_df = pd.read_csv(cert_path, low_memory=False)

        epc_df['postcode_clean'] = epc_df['POSTCODE'].astype(str).str.replace(' ', '').str.upper()
        epc_df['match_key'] = epc_df.apply(
            lambda row: normalize_address_components(
                row.get('ADDRESS1', ''),
                row.get('ADDRESS2', ''),
                row['postcode_clean']
            ),
            axis=1
        )

        matched = epc_df[epc_df['match_key'].isin(price_df['match_key'])]
        print('Done')

        if not matched.empty:
            # Save matched rows to disk
            out_path = os.path.join(output_dir, f"{subfolder}.parquet")
            matched.to_parquet(out_path, index=False)

    except Exception as e:
        print(f"❌ Error processing {cert_path}: {e}")

Done


In [3]:
import os
import pandas as pd
import re
from tqdm import tqdm


# --- Normalization Function ---
def normalize_address_components(*components):
    combined = ' '.join([str(c) for c in components if pd.notna(c)])
    normalized = (
        combined.upper()
        .replace(',', '')
        .replace('.', '')
        .replace('-', ' ')
    )
    normalized = re.sub(r'\s+', ' ', normalized)
    return normalized.strip()

# --- Normalize price_df ---
price_df['postcode_clean'] = price_df['Postcode'].astype(str).str.replace(' ', '').str.upper()
price_df['match_key'] = price_df.apply(
    lambda row: normalize_address_components(
        row.get('primary_addressable_object_name', ''),
        row.get('street', ''),
        row['postcode_clean']
    ),
    axis=1
)

In [6]:
import os
import pandas as pd

matched_dir = 'matched_epc_rows'

# Get all individual parquet files
parquet_files = [
    os.path.join(matched_dir, f)
    for f in os.listdir(matched_dir)
    if f.endswith('.parquet')
]

# Read and combine
matched_dfs = [pd.read_parquet(f) for f in parquet_files]

if matched_dfs:
    total_matched = pd.concat(matched_dfs, ignore_index=True)
    print(f"✅ Loaded total matched rows: {len(total_matched):,}")
    display(total_matched.head())
else:
    print("⚠ No parquet files found in matched_epc_rows.")

✅ Loaded total matched rows: 4,209,390


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE,REPORT_TYPE,postcode_clean,match_key
0,1451865729102016060917325243560118,"54, Chatham Road",None,None,TS24 8QQ,134825478,D,B,61,85,...,England and Wales: 1930-1949,2016-06-09 17:32:52,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100,TS248QQ,54 CHATHAM ROAD TS248QQ
1,978835396212019052007505697910817,"19, Wainwright Walk",None,None,TS25 1XA,8777671178,D,C,65,79,...,England and Wales: 1967-1975,2019-05-20 07:50:56,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100,TS251XA,19 WAINWRIGHT WALK TS251XA
2,1341712688132019081910310554978408,"116, Milton Road",None,None,TS26 8DX,8864447378,D,C,56,77,...,England and Wales: 1900-1929,2019-08-19 10:31:05,rental (private),NaN,NaN,1.001100e+11,Address Matched,100,TS268DX,116 MILTON ROAD TS268DX
3,1186183659802014081111373821640768,"18, Brafferton Street",None,None,TS26 8LH,4895056278,D,B,57,90,...,England and Wales: 1900-1929,2014-08-11 11:37:38,rental (private),7.0,0.0,1.001100e+11,Address Matched,100,TS268LH,18 BRAFFERTON STREET TS268LH
4,1566486539802017081008512757330418,"132, Kesteven Road",None,None,TS25 2NJ,862833578,C,B,70,85,...,England and Wales: 1967-1975,2017-08-10 08:51:27,owner-occupied,NaN,NaN,1.001100e+11,Address Matched,100,TS252NJ,132 KESTEVEN ROAD TS252NJ


In [11]:
price_df.columns

Index(['Transfer Date', 'Postcode', 'primary_addressable_object_name',
       'secondary_addressable_object_name', 'street', 'Town/City', 'District',
       'County', 'Price (Thousands)', 'Freehold Tenure', 'Leasehold Tenure',
       'Old Build', 'New Build', 'Detached', 'Flat', 'Other Property Type',
       'Semi-detached', 'Terraced', 'Year', 'Month', 'Quarter',
       'Day of the Week', 'Postcode Area', 'Postcode District',
       'Area Code Average Price (Thousands)',
       'Town/City Average Price (Thousands)',
       'District Average Price (Thousands)',
       'County Average Price (Thousands)', 'LAT', 'LONG', 'OSEAST1M',
       'OSNRTH1M', 'postcode_clean', 'match_key'],
      dtype='object')

In [4]:
trim_price = price_df.drop(columns=['Postcode', 'primary_addressable_object_name', 'secondary_addressable_object_name', 'street', 'District', 'Town/City'], axis=1)
trim_price.columns

Index(['Transfer Date', 'County', 'Price (Thousands)', 'Freehold Tenure',
       'Leasehold Tenure', 'Old Build', 'New Build', 'Detached', 'Flat',
       'Other Property Type', 'Semi-detached', 'Terraced', 'Year', 'Month',
       'Quarter', 'Day of the Week', 'Postcode Area', 'Postcode District',
       'Area Code Average Price (Thousands)',
       'Town/City Average Price (Thousands)',
       'District Average Price (Thousands)',
       'County Average Price (Thousands)', 'LAT', 'LONG', 'OSEAST1M',
       'OSNRTH1M', 'postcode_clean', 'match_key'],
      dtype='object')

In [5]:
trim_price.to_csv('Full_Date.csv', index=False)

In [14]:
 total_matched.columns

Index(['LMK_KEY', 'ADDRESS1', 'ADDRESS2', 'ADDRESS3', 'POSTCODE',
       'BUILDING_REFERENCE_NUMBER', 'CURRENT_ENERGY_RATING',
       'POTENTIAL_ENERGY_RATING', 'CURRENT_ENERGY_EFFICIENCY',
       'POTENTIAL_ENERGY_EFFICIENCY', 'PROPERTY_TYPE', 'BUILT_FORM',
       'INSPECTION_DATE', 'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY',
       'LODGEMENT_DATE', 'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT',
       'ENVIRONMENT_IMPACT_POTENTIAL', 'ENERGY_CONSUMPTION_CURRENT',
       'ENERGY_CONSUMPTION_POTENTIAL', 'CO2_EMISSIONS_CURRENT',
       'CO2_EMISS_CURR_PER_FLOOR_AREA', 'CO2_EMISSIONS_POTENTIAL',
       'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
       'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL',
       'HOT_WATER_COST_CURRENT', 'HOT_WATER_COST_POTENTIAL',
       'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF', 'MAINS_GAS_FLAG', 'FLOOR_LEVEL',
       'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT', 'MAIN_HEATING_CONTROLS',
       'MULTI_GLAZE_PROPORTION', 'GLAZED_TYPE', 'GLAZED_AREA',
      

In [7]:
matched_trimmed = total_matched.drop(
    columns=[
        'LMK_KEY', 'ADDRESS3', 'BUILDING_REFERENCE_NUMBER', 'INSPECTION_DATE',
        'LOCAL_AUTHORITY', 'CONSTITUENCY', 'COUNTY', 'LODGEMENT_DATE',
        'TRANSACTION_TYPE', 'ENVIRONMENT_IMPACT_CURRENT', 'ENVIRONMENT_IMPACT_POTENTIAL',
        'ENERGY_CONSUMPTION_CURRENT', 'ENERGY_CONSUMPTION_POTENTIAL',
        'CO2_EMISSIONS_CURRENT', 'CO2_EMISS_CURR_PER_FLOOR_AREA',
        'CO2_EMISSIONS_POTENTIAL', 'LIGHTING_COST_CURRENT', 'LIGHTING_COST_POTENTIAL',
        'HEATING_COST_CURRENT', 'HEATING_COST_POTENTIAL', 'HOT_WATER_COST_CURRENT',
        'HOT_WATER_COST_POTENTIAL', 'ENERGY_TARIFF', 'FLOOR_DESCRIPTION',
        'WINDOWS_DESCRIPTION', 'WALLS_DESCRIPTION', 'SECONDHEAT_DESCRIPTION',
        'ROOF_DESCRIPTION', 'MAINHEAT_DESCRIPTION', 'MAINHEATCONT_DESCRIPTION',
        'LIGHTING_DESCRIPTION', 'HEAT_LOSS_CORRIDOR', 'UNHEATED_CORRIDOR_LENGTH',
        'PHOTO_SUPPLY', 'MECHANICAL_VENTILATION', 'ADDRESS', 'LOCAL_AUTHORITY_LABEL',
        'CONSTITUENCY_LABEL', 'POSTTOWN', 'LODGEMENT_DATETIME', 'TENURE',
        'FIXED_LIGHTING_OUTLETS_COUNT', 'LOW_ENERGY_FIXED_LIGHT_COUNT',
        'UPRN', 'UPRN_SOURCE', 'REPORT_TYPE', 'postcode_clean'
    ]
)

In [16]:
merge_df = matched_trimmed.merge(trim_price, how='left', on=['match_key'])

In [18]:
merge_df.shape

(4864524, 74)

In [19]:
merge_df.to_csv('merge.csv', index=False)

In [25]:
summary_df = pd.DataFrame({
    'Column': merge_df.columns,
    'Dtype': merge_df.dtypes.values,
    'Missing': merge_df.isna().sum().values,
    'Percent': (((merge_df.isna().sum().values)/4864524)*100).round(1)
})

print(summary_df)

                     Column    Dtype  Missing  Percent
0                  ADDRESS1   object        1      0.0
1                  ADDRESS2   object  4543251     93.4
2                  POSTCODE   object        0      0.0
3     CURRENT_ENERGY_RATING   object        0      0.0
4   POTENTIAL_ENERGY_RATING   object        0      0.0
..                      ...      ...      ...      ...
69                      LAT  float64       24      0.0
70                     LONG  float64       24      0.0
71                 OSEAST1M  float64       24      0.0
72                 OSNRTH1M  float64       24      0.0
73           postcode_clean   object        0      0.0

[74 rows x 4 columns]


In [26]:
summary_df.to_csv('column_summary.csv', index=False)

In [61]:
save=merge_df

In [27]:
drop_cols = ['ADDRESS2', 'FLAT_TOP_STOREY', 'FLAT_STOREY_COUNT','HOT_WATER_ENV_EFF', 'FLOOR_ENERGY_EFF', 'WALLS_ENV_EFF', 'SHEATING_ENERGY_EFF','FLOOR_ENV_EFF','SHEATING_ENV_EFF']
merge_df.drop(columns=drop_cols, inplace=True, errors='ignore')

# Impute with mode
mode_impute_cols = ['MAINS_GAS_FLAG', 'MAIN_HEATING_CONTROLS','GLAZED_TYPE', 'LOW_ENERGY_LIGHTING','SOLAR_WATER_HEATING_FLAG', 'MAIN_FUEL', 'GLAZED_AREA', 'ROOF_ENERGY_EFF', 'ROOF_ENV_EFF']

for col in mode_impute_cols:
    if col in merge_df.columns:
        mode_series = merge_df[col].mode(dropna=True)
        if not mode_series.empty:
            merge_df[col].fillna(mode_series[0], inplace=True)

median_impute_cols = ['MULTI_GLAZE_PROPORTION', '', 'EXTENSION_COUNT', 'NUMBER_HABITABLE_ROOMS', 'NUMBER_HEATED_ROOMS', 'FLOOR_HEIGHT']

for col in median_impute_cols:
    if col in merge_df.columns:
        merge_df[col].fillna(merge_df[col].median(), inplace=True)

# Impute with 0
zero_fill_cols = ['EXTENSION_COUNT', 'WIND_TURBINE_COUNT','NUMBER_OPEN_FIREPLACES']
for col in zero_fill_cols:
    if col in merge_df.columns:
        merge_df[col].fillna(0, inplace=True)

# Fill with 'Unknown' for specific columns
unknown_bin_cols = ['FLOOR_LEVEL']
for col in unknown_bin_cols:
    if col in merge_df.columns:
        merge_df[col] = merge_df[col].fillna('Unknown')

# DROP ROWS that still contain NaNs in important predictive columns
critical_cols = ['BUILT_FORM', 'LAT', 'LONG', 'OSEAST1M', 'OSNRTH1M','LIGHTING_ENV_EFF', 'LIGHTING_ENERGY_EFF', 'MAINHEATC_ENV_EFF','MAINHEATC_ENERGY_EFF', 'MAINHEAT_ENV_EFF', 'MAINHEAT_ENERGY_EFF','WALLS_ENERGY_EFF',  'WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF', 'HOT_WATER_ENERGY_EFF', 'HOTWATER_DESCRIPTION', 'ADDRESS1']

merge_df.dropna(subset=[col for col in critical_cols if col in merge_df.columns], inplace=True)

# Final check
missing_summary = merge_df.isnull().sum()
print("Remaining missing values:\n", missing_summary[missing_summary > 0])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5824\1154077277.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge_df[col].fillna(mode_series[0], inplace=True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_5824\1154077277.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

Remaining missing values:
 CONSTRUCTION_AGE_BAND    44929
dtype: int64


In [28]:
merge_df['CONSTRUCTION_AGE_BAND'] = merge_df['CONSTRUCTION_AGE_BAND'].fillna('Unknown')
merge_df.shape

(4857993, 65)

In [31]:
turn_binary = merge_df
turn_binary.shape

(4857993, 65)

In [38]:
summary_df1 = pd.DataFrame({
    'Column': turn_binary.columns,
    'Dtype': turn_binary.dtypes.values,
    'Example': turn_binary.iloc[0].values

})

summary_df1.to_csv('dtype_inq.csv', index=False)

In [86]:
turn_binary = turn_binary.drop(columns=[
    'ADDRESS1','POSTCODE','CURRENT_ENERGY_RATING','POTENTIAL_ENERGY_RATING',
    'BUILT_FORM','FLOOR_LEVEL','GLAZED_TYPE','GLAZED_AREA',
    'MAIN_FUEL','PROPERTY_TYPE','match_key','County',
    'postcode_clean','Postcode District','Postcode Area'
])

turn_binary['MAINS_GAS_FLAG'] = turn_binary['MAINS_GAS_FLAG'].map({'Y': 1, 'N': 0})
turn_binary['SOLAR_WATER_HEATING_FLAG'] = turn_binary['SOLAR_WATER_HEATING_FLAG'].map({'Y': 1, 'N': 0})

scale = ['HOT_WATER_ENERGY_EFF', 'WINDOWS_ENERGY_EFF','WINDOWS_ENV_EFF', 'WALLS_ENERGY_EFF', 'ROOF_ENERGY_EFF','ROOF_ENV_EFF', 'MAINHEAT_ENERGY_EFF', 'MAINHEAT_ENV_EFF','MAINHEATC_ENERGY_EFF', 'MAINHEATC_ENV_EFF', 'LIGHTING_ENERGY_EFF','LIGHTING_ENV_EFF']

for i in scale:
    turn_binary[i] = turn_binary[i].map({'Very Poor': 0, 'Poor': 1, 'Average': 2, 'Good': 3,'Very Good': 4})

In [50]:
print(df['CONSTRUCTION_AGE_BAND'].unique())

['England and Wales: 1930-1949' 'England and Wales: 1967-1975'
 'England and Wales: 1900-1929' 'NO DATA!' 'England and Wales: 1950-1966'
 'England and Wales: before 1900' 'England and Wales: 2007 onwards'
 'England and Wales: 1983-1990' 'England and Wales: 1976-1982' 'INVALID!'
 '2019' 'England and Wales: 1996-2002' 'England and Wales: 2003-2006'
 '2020' 'England and Wales: 1991-1995' None '2021'
 'England and Wales: 2007-2011' '2017' '2022'
 'England and Wales: 2012 onwards' '2023' '2024' '2016' '2025'
 'England and Wales: 2012-2021']


In [87]:
import numpy as np

def clean_age_band(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip()

    # Remove prefix
    if val.startswith('England and Wales:'):
        val = val.replace('England and Wales:', '').strip()

    # Handle year strings
    if val.isdigit():
        year = int(val)
        if year <= 1929:
            return '1900-1929'
        elif year <= 1949:
            return '1930-1949'
        elif year <= 1966:
            return '1950-1966'
        elif year <= 1975:
            return '1967-1975'
        elif year <= 1982:
            return '1976-1982'
        elif year <= 1990:
            return '1983-1990'
        elif year <= 1995:
            return '1991-1995'
        elif year <= 2002:
            return '1996-2002'
        elif year <= 2006:
            return '2003-2006'
        elif year <= 2011:
            return '2007-2011'
        elif year <= 2021:
            return '2012-2021'
        else:
            return '2022 onwards'

    # Normalize known bad values
    if val in ['NO DATA!', 'INVALID!', 'None']:
        return np.nan

    # Leave standard bucketed strings
    return val

# Apply cleaning
turn_binary['CONSTRUCTION_AGE_BAND_CLEAN'] = turn_binary['CONSTRUCTION_AGE_BAND'].apply(clean_age_band)

# Now map to ordinal
age_band_map = {
    'before 1900': 0,
    '1900-1929': 1,
    '1930-1949': 2,
    '1950-1966': 3,
    '1967-1975': 4,
    '1976-1982': 5,
    '1983-1990': 6,
    '1991-1995': 7,
    '1996-2002': 8,
    '2003-2006': 9,
    '2007-2011': 10,
    '2012-2021': 11,
    '2022 onwards': 12
}

turn_binary['CONSTRUCTION_AGE_BAND_ENCODED'] turn_binary['CONSTRUCTION_AGE_BAND_CLEAN'].map(age_band_map)

SyntaxError: invalid syntax (1532537879.py, line 67)

In [90]:
turn_binary.dtypes

CURRENT_ENERGY_EFFICIENCY                int64
POTENTIAL_ENERGY_EFFICIENCY              int64
TOTAL_FLOOR_AREA                       float64
MAINS_GAS_FLAG                         float64
MAIN_HEATING_CONTROLS                   object
MULTI_GLAZE_PROPORTION                 float64
EXTENSION_COUNT                        float64
NUMBER_HABITABLE_ROOMS                 float64
NUMBER_HEATED_ROOMS                    float64
LOW_ENERGY_LIGHTING                    float64
NUMBER_OPEN_FIREPLACES                 float64
HOTWATER_DESCRIPTION                    object
HOT_WATER_ENERGY_EFF                   float64
WINDOWS_ENERGY_EFF                     float64
WINDOWS_ENV_EFF                        float64
WALLS_ENERGY_EFF                       float64
ROOF_ENERGY_EFF                        float64
ROOF_ENV_EFF                           float64
MAINHEAT_ENERGY_EFF                    float64
MAINHEAT_ENV_EFF                       float64
MAINHEATC_ENERGY_EFF                   float64
MAINHEATC_ENV

In [95]:
conv = turn_binary[turn_binary['MAIN_HEATING_CONTROLS'] != '%%MAINHEATCONTROL%%']

In [100]:
conv['MAIN_HEATING_CONTROLS'] = conv['MAIN_HEATING_CONTROLS'].astype('float64')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5824\3640805052.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conv['MAIN_HEATING_CONTROLS'] = conv['MAIN_HEATING_CONTROLS'].astype('float64')


In [101]:
conv.dtypes

CURRENT_ENERGY_EFFICIENCY                int64
POTENTIAL_ENERGY_EFFICIENCY              int64
TOTAL_FLOOR_AREA                       float64
MAINS_GAS_FLAG                         float64
MAIN_HEATING_CONTROLS                  float64
MULTI_GLAZE_PROPORTION                 float64
EXTENSION_COUNT                        float64
NUMBER_HABITABLE_ROOMS                 float64
NUMBER_HEATED_ROOMS                    float64
LOW_ENERGY_LIGHTING                    float64
NUMBER_OPEN_FIREPLACES                 float64
HOTWATER_DESCRIPTION                    object
HOT_WATER_ENERGY_EFF                   float64
WINDOWS_ENERGY_EFF                     float64
WINDOWS_ENV_EFF                        float64
WALLS_ENERGY_EFF                       float64
ROOF_ENERGY_EFF                        float64
ROOF_ENV_EFF                           float64
MAINHEAT_ENERGY_EFF                    float64
MAINHEAT_ENV_EFF                       float64
MAINHEATC_ENERGY_EFF                   float64
MAINHEATC_ENV

In [103]:
conv = conv.drop(columns='HOTWATER_DESCRIPTION')

In [106]:
conv = conv.drop(columns=['CONSTRUCTION_AGE_BAND_CLEAN','CONSTRUCTION_AGE_BAND'])

In [107]:
conv.shape

(4857909, 49)

In [108]:
conv.to_csv('EPC_Fin.csv', index=False)